# Derivation

$$
x_k := \text{argmax } p(y_k | c, x_k)p(x_k) = \text{argmax} \left( log\ p(y_k | c, x_k) + log\ p(x_k) \right)
$$

where $p(y_k | c, x_k) = Bernoulli(y_k; \psi(cx_k))$ and $p(x_k) = N(0, \sigma_x^2)$. $\psi$ is the sigmoid function.

Then, we obtain the gradient descent update for $x_k$:
$$
x_k := x_k + \gamma_x (c^\intercal (y_k (1 - \psi(cx_k)) - (1 - y_k)\psi(cx_k)) - x_k / \sigma_x^2)
$$

Similarly, the update for $c$ is:

$$
c := c + \gamma_c \nabla log \pi(c) = c + \gamma_c ((y_k(1 - \psi(cx_k)) - (1-y_k)\psi(cx_k)) x_k^\intercal - c / \sigma_c^2)
$$

Incorporating Langevin Monte Carlo (LMC),

$$
\tilde{c} := c + \gamma_c ((y_k(1 - \psi(cx_k)) - (1-y_k)\psi(cx_k)) x_k^\intercal - c / \sigma_c^2) + \sqrt{2 \gamma_c} \xi
$$

According to the Metropolis-Hastings algorithm,
$$
\alpha := min\left(1, \frac{\pi(\tilde{c})q(c | \tilde{c})}{\pi(c)q(\tilde{c}|c)} \right)
$$

where
$$
log q(\tilde{c}|c) = -\frac{1}{4\gamma_c} ||\tilde{c} - c - \gamma_c \nabla log \pi(c)||_2^2
$$

and
$$
log \pi(c) = ||y_k log(\psi(cx_k)) + (1-y_k) log(1-\psi(cx_k))||_1^1 - \frac{1}{2\sigma_c^2}||c||_2^2 + const
$$
$$
= ||y_k log\psi(cx_k) + (1-y_k) log\psi(-cx_k)||_1^1 - \frac{1}{2\sigma_c^2}||c||_2^2 + const
$$

In [1]:
import numpy as np

In [2]:
Y = np.array([[1, 1, 0, 0], [1, 0, 1, 0], [0, 1, 0, 1], [0, 0, 1, 1]])
C = np.random.normal(0, 1, (4, 2))
X = np.random.normal(0, 1, (2, 4))

In [3]:
def sigmoid(z):
    return 1/(1 + np.exp(-z))

In [4]:
gamma = 0.01
sigma = 10.0
for i in range(1000):
    X = X + gamma * (C.T @ (Y * (1 - sigmoid(C @ X)) - (1 - Y) * sigmoid(C @ X)) - (X / (sigma ** 2)))
    C = C + gamma * ((Y * (1 - sigmoid(C @ X)) - (1 - Y) * sigmoid(C @ X)) @ X.T - (C / (sigma ** 2)))
print(sigmoid(C @ X))
print(C)
print(X)

[[0.99201666 0.99433331 0.00558017 0.00919788]
 [0.98986955 0.00913576 0.9913542  0.00754601]
 [0.00941501 0.99141028 0.00812236 0.99301731]
 [0.00399306 0.00808901 0.99198774 0.99552854]]
[[ 1.03588176  2.47239243]
 [-2.3781237   0.95177459]
 [ 2.41179306 -0.96909918]
 [-0.79838346 -2.66873235]]
[[-0.98152615  2.403997   -2.42617075  1.10833048]
 [ 2.36173211  1.082842   -1.07980997 -2.35708591]]


In [5]:
import sys
sys.path.append('../Library')

from Modules.BernoulliDF import BernoulliDF as DictionaryFilter

2024-08-14 15:47:38.900856: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-14 15:47:38.921597: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-14 15:47:38.921608: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-14 15:47:38.922359: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-14 15:47:38.926318: I tensorflow/core/platform/cpu_feature_guar

In [6]:
import tensorflow as tf
import tensorflow_ranking as tfr
import pandas as pd
import numpy as np

In [7]:
model = DictionaryFilter(4, 4, 2, 0.1, 10.0, 0.1, 10.0)

Y = tf.constant([[1, 1, 0, 0], [1, 0, 1, 0], [0, 1, 0, 1], [0, 0, 1, 1]], dtype=tf.float64)
k = tf.constant(0)
for i in range(1000):
    Y_hat = model(Y, k)

print(model.C.numpy())
print(model.X.numpy())
print(Y_hat.numpy())

2024-08-14 15:47:39.953026: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-14 15:47:39.968056: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-14 15:47:39.968093: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-14 15:47:39.969698: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-14 15:47:39.969726: I external/local_xla/xla/stream_executor

[[  2.11854784   5.93900714]
 [  1.34312121  -6.87091708]
 [ -0.70662113   1.54607748]
 [-13.02010853   0.78407878]]
[[ 3.66805907  0.50508478 -0.50508478 -3.66805907]
 [-0.53467541  2.22095676 -2.22095676  0.53467541]]
[[9.90003936e-01 9.99999359e-01 6.40943237e-07 9.99606378e-03]
 [9.99815994e-01 4.64817908e-07 9.99999535e-01 1.84005967e-04]
 [3.17198848e-02 9.55928409e-01 4.40715910e-02 9.68280115e-01]
 [1.19307274e-21 7.88518203e-03 9.92114818e-01 1.00000000e+00]]


In [8]:
# Read the gowalla dataset
train_data = pd.read_csv('../Data/Binary/toy-dataset/train.csv')
test_data = pd.read_csv('../Data/Binary/toy-dataset/test.csv')

In [9]:
# Print the shape of the data
print(train_data.shape)
print(test_data.shape)

(399920, 3)
(99980, 3)


In [10]:
NUM_MOVIES = train_data['Movie Index'].max() + 1
NUM_USERS = train_data['User Index'].max() + 1
NUM_FACTORS = 4
BATCH_SIZE = 250
GAMMA_C = 0.0001
GAMMA_X = 0.0001
SIGMA_C = 1.0
SIGMA_X = 1.0

In [11]:
# Convert train data to sparse tensor
train_sparse_tensor = tf.sparse.SparseTensor(
    indices=train_data[['Movie Index', 'User Index']].values,
    values=train_data['Rating'].values,
    dense_shape=[NUM_MOVIES, NUM_USERS]
)
train_sparse_tensor = tf.sparse.reorder(train_sparse_tensor)

# Convert test data to sparse tensor
test_sparse_tensor = tf.sparse.SparseTensor(
    indices=test_data[['Movie Index', 'User Index']].values,
    values=test_data['Rating'].values,
    dense_shape=[NUM_MOVIES, NUM_USERS]
)
test_sparse_tensor = tf.sparse.reorder(test_sparse_tensor)

# Create dataset
def data_generator():
    train_slices = tf.sparse.split(sp_input=train_sparse_tensor, num_split=NUM_USERS // BATCH_SIZE, axis=1)
    test_slices = tf.sparse.split(sp_input=test_sparse_tensor, num_split=NUM_USERS // BATCH_SIZE, axis=1)
    for i in range(NUM_USERS // BATCH_SIZE):
        yield (tf.sparse.to_dense(train_slices[i]), tf.sparse.to_dense(test_slices[i]))

dataset = tf.data.Dataset.from_generator(
    data_generator, 
    output_signature=(
        tf.TensorSpec(shape=[NUM_MOVIES, None], dtype=tf.float64),
        tf.TensorSpec(shape=[NUM_MOVIES, None], dtype=tf.float64)
    )
)

In [12]:
from tqdm import tqdm

# Create the model
model = DictionaryFilter(NUM_MOVIES, NUM_USERS, NUM_FACTORS, GAMMA_X, SIGMA_X, GAMMA_C, SIGMA_C)
recall = tf.keras.metrics.Recall(top_k=20)
ndcg = tfr.keras.metrics.NDCGMetric(topn=20)
# min_recall = tf.keras.metrics.Recall(top_k=20) ~ 0.02 on toy-dataset
# min_ndcg = tfr.keras.metrics.NDCGMetric(topn=20) ~ 0.5 on toy-dataset
# max_recall = tf.keras.metrics.Recall(top_k=20) ~ 0.2 on toy-dataset
# max_ndcg = tfr.keras.metrics.NDCGMetric(topn=20) ~ 1.0 on toy-dataset

# Train the model
progress_bar = tqdm(range(100), desc="Training Progress", total=100)
for epoch in progress_bar:
    k = tf.Variable(0, trainable=False)
    for train_batch, test_batch in dataset:
        Y_hat = model(train_batch, k)
        k.assign_add(train_batch.shape[1])

        # Compute the metrics
        Y_test = tf.multiply(Y_hat, tf.subtract(1.0, train_batch))
        recall.update_state(tf.transpose(Y_test), tf.transpose(test_batch))
        ndcg.update_state(tf.transpose(Y_test), tf.transpose(test_batch))

    # Update the progress bar description
    progress_bar.set_postfix({"Recall": recall.result().numpy(), "NDCG": ndcg.result().numpy()})

    recall.reset_states()
    ndcg.reset_states()

Training Progress:   9%|▉         | 88/1000 [00:09<01:38,  9.31it/s, Recall=0.0333, NDCG=0.785]


KeyboardInterrupt: 